### Open Flights Data Wrangling

To practice, you are going to wrangle data from OpenFlights.  You can read about it here: 

http://openflights.org/data.html

This includes three main files, one for each airport, one for each airline, and one for each route.  They can be merged or joined with the appropriate fields.  I have modified the files slightly to include a header row in the .dat files, which makes it a bit easier for you.  

You are required to work through the problems below.  This may take some time.  Be persistent, and ask questions or seek help as needed.  

In [51]:
import pandas as pd
import numpy as np

In [52]:
# These files use \N as a missing value indicator.  When reading the CSVs, we will tell
# it to use that value as missing or NA.  The double backslash is required because
# otherwise it will interpret \N as a carriage return. 

# Read in the airports data.
airports = pd.read_csv("airports.dat", header=None, na_values='\\N')
airports.columns = ["id", "name", "city", "country", "iata", "icao", "latitude", "longitude", "altitude","timezone", "dst", "tz", "type", "source"]

# Read in the airlines data.
airlines = pd.read_csv("airlines.dat", header=None, na_values='\\N')
airlines.columns = ["id", "name", "alias", "iata", "icao", "callsign", "country", "active"]

# Read in the routes data.
routes = pd.read_csv("routes.dat", header=None, na_values='\\N')
routes.columns = ["airline", "airline_id", "source", "source_id", "dest", "dest_id", "codeshare", "stops", "equipment"]

1) Start by seeing what's in the data.  What columns are there?  What data types are the columns?  

Remember, 'object' means it is a string, while the numerical values can be floats or ints.  Sometimes you will have problems if it reads numeric data in as strings.  If that happens, you can use the function .astype() to convert it.  Look it up in the pandas API to get more details

In [70]:
airports.dtypes

id             int64
name          object
city          object
country       object
iata          object
icao          object
latitude     float64
longitude    float64
altitude       int64
timezone     float64
dst           object
tz            object
type          object
source        object
dtype: object

In [72]:
airlines.dtypes

id           int64
name        object
alias       object
iata        object
icao        object
callsign    object
country     object
active      object
dtype: object

In [74]:
routes.dtypes

airline        object
airline_id    float64
source         object
source_id     float64
dest           object
dest_id       float64
codeshare      object
stops           int64
equipment      object
dtype: object

2) Select just the routes that go to or from Lexington Bluegrass Airport, and store them in their own dataframe.  

The airport code is LEX.  You should have a much smaller dataframe.  How many inbound routes and how many outbound routes are there? 

In [56]:
rlex = routes[(routes['source'] == 'LEX') | (routes['dest'] == 'LEX')]
rlex

,airline,airline_id,source,source_id,dest,dest_id,codeshare,stops,equipment
3569,9E,3976.0,ATL,3682.0,LEX,4017.0,NaN,0,CRJ
3588,9E,3976.0,LEX,4017.0,ATL,3682.0,NaN,0,CRJ
4953,AA,24.0,CLT,3876.0,LEX,4017.0,NaN,0,CR7
5247,AA,24.0,DFW,3670.0,LEX,4017.0,Y,0,ERD ER4
5763,AA,24.0,LEX,4017.0,CLT,3876.0,Y,0,CR7 CRJ
5764,AA,24.0,LEX,4017.0,DFW,3670.0,Y,0,ERD ER4
5765,AA,24.0,LEX,4017.0,ORD,3830.0,Y,0,ERD ER4
6283,AA,24.0,ORD,3830.0,LEX,4017.0,Y,0,ERD ER4
9097,AF,137.0,ATL,3682.0,LEX,4017.0,Y,0,CR9 M88 CRJ 717
9641,AF,137.0,LEX,4017.0,ATL,3682.0,Y,0,CRJ CR9


In [57]:
rout=rlex['source'].str.count('LEX').sum()
rout

20

In [58]:
rin=rlex['dest'].str.count('LEX').sum()
rin

20

3) Now let's look at which airlines operate in and out of Lexington.  To do this, you need to merge the airline dataframe to the route dataframe.  

How many routes does each airline have?  The value_counts() method may be useful for answering this question.  

In [63]:
rlines = pd.merge(airlines, rlex, left_on='id', right_on='airline_id')
rlines

,id,name,alias,iata,icao,callsign,country,active,airline,airline_id,source,source_id,dest,dest_id,codeshare,stops,equipment
0,24,American Airlines,NaN,AA,AAL,AMERICAN,United States,Y,AA,24.0,CLT,3876.0,LEX,4017.0,NaN,0,CR7
1,24,American Airlines,NaN,AA,AAL,AMERICAN,United States,Y,AA,24.0,DFW,3670.0,LEX,4017.0,Y,0,ERD ER4
2,24,American Airlines,NaN,AA,AAL,AMERICAN,United States,Y,AA,24.0,LEX,4017.0,CLT,3876.0,Y,0,CR7 CRJ
3,24,American Airlines,NaN,AA,AAL,AMERICAN,United States,Y,AA,24.0,LEX,4017.0,DFW,3670.0,Y,0,ERD ER4
4,24,American Airlines,NaN,AA,AAL,AMERICAN,United States,Y,AA,24.0,LEX,4017.0,ORD,3830.0,Y,0,ERD ER4
5,24,American Airlines,NaN,AA,AAL,AMERICAN,United States,Y,AA,24.0,ORD,3830.0,LEX,4017.0,Y,0,ERD ER4
6,35,Allegiant Air,NaN,G4,AAY,ALLEGIANT,United States,Y,G4,35.0,FLL,3533.0,LEX,4017.0,NaN,0,M80
7,35,Allegiant Air,NaN,G4,AAY,ALLEGIANT,United States,Y,G4,35.0,LEX,4017.0,FLL,3533.0,NaN,0,M80
8,35,Allegiant Air,NaN,G4,AAY,ALLEGIANT,United States,Y,G4,35.0,LEX,4017.0,PGD,7056.0,NaN,0,M80
9,35,Allegiant Air,NaN,G4,AAY,ALLEGIANT,United States,Y,G4,35.0,LEX,4017.0,PIE,3617.0,NaN,0,320


In [64]:
rlines['name'].value_counts()

Delta Air Lines             10
Allegiant Air                8
US Airways                   6
American Airlines            6
United Airlines              4
Air France                   2
Pinnacle Airlines            2
KLM Royal Dutch Airlines     2
Name: name, dtype: int64

4) It looks like there are some international airlines with Lexington routes.  To look at how many routes they have, create a new column in your dataframe called 'International', which is set to Y for an overseas airline and N for a domestic airline.  Calculate the percent of routes with an overseas airline.  

In [65]:
def clean_ar(value):
    if isinstance(value, str):
        end = value.find('United States')
        if end == -1:
            return 'Y'
        else:
            return 'N'
rlines['international'] = rlines['country'].map(clean_ar)

rlines

,id,name,alias,iata,icao,callsign,country,active,airline,airline_id,source,source_id,dest,dest_id,codeshare,stops,equipment,international
0,24,American Airlines,NaN,AA,AAL,AMERICAN,United States,Y,AA,24.0,CLT,3876.0,LEX,4017.0,NaN,0,CR7,N
1,24,American Airlines,NaN,AA,AAL,AMERICAN,United States,Y,AA,24.0,DFW,3670.0,LEX,4017.0,Y,0,ERD ER4,N
2,24,American Airlines,NaN,AA,AAL,AMERICAN,United States,Y,AA,24.0,LEX,4017.0,CLT,3876.0,Y,0,CR7 CRJ,N
3,24,American Airlines,NaN,AA,AAL,AMERICAN,United States,Y,AA,24.0,LEX,4017.0,DFW,3670.0,Y,0,ERD ER4,N
4,24,American Airlines,NaN,AA,AAL,AMERICAN,United States,Y,AA,24.0,LEX,4017.0,ORD,3830.0,Y,0,ERD ER4,N
5,24,American Airlines,NaN,AA,AAL,AMERICAN,United States,Y,AA,24.0,ORD,3830.0,LEX,4017.0,Y,0,ERD ER4,N
6,35,Allegiant Air,NaN,G4,AAY,ALLEGIANT,United States,Y,G4,35.0,FLL,3533.0,LEX,4017.0,NaN,0,M80,N
7,35,Allegiant Air,NaN,G4,AAY,ALLEGIANT,United States,Y,G4,35.0,LEX,4017.0,FLL,3533.0,NaN,0,M80,N
8,35,Allegiant Air,NaN,G4,AAY,ALLEGIANT,United States,Y,G4,35.0,LEX,4017.0,PGD,7056.0,NaN,0,M80,N
9,35,Allegiant Air,NaN,G4,AAY,ALLEGIANT,United States,Y,G4,35.0,LEX,4017.0,PIE,3617.0,NaN,0,320,N


In [66]:
prcnt = ((rlines['international'].str.count('Y').sum()) / (rlines['international'].value_counts().sum())) * 100
prcnt

10.0

5) Actually, it looks like a bunch of these routes are codeshares.  That means they are marketed by this airline, but operated by a different airline.  See the note in the data documentation on openflights.org/data.  The implication of this is that there are duplicates.

Can you figure out which ones are duplicates?  Can you then create a dataframe with only the unique routes?  How many unique inbound and outbound routes are there? 

Remember, someone has to operate the flight, so if all the routes to/from a particular airport are listed as codeshares, then something is funny...

It is also possible that more than one airline actually operates a route between the same two airports. (Having this sort of competition generally means that you will get better fares as a traveler.)  It may not be obvious what is actually in the data set, so dig or do external research as needed.  

In [89]:
rports = rlines.drop_duplicates(['source', 'dest', 'codeshare'])
rports

,id,name,alias,iata,icao,callsign,country,active,airline,airline_id,source,source_id,dest,dest_id,codeshare,stops,equipment,international
0,24,American Airlines,NaN,AA,AAL,AMERICAN,United States,Y,AA,24.0,CLT,3876.0,LEX,4017.0,NaN,0,CR7,N
1,24,American Airlines,NaN,AA,AAL,AMERICAN,United States,Y,AA,24.0,DFW,3670.0,LEX,4017.0,Y,0,ERD ER4,N
2,24,American Airlines,NaN,AA,AAL,AMERICAN,United States,Y,AA,24.0,LEX,4017.0,CLT,3876.0,Y,0,CR7 CRJ,N
3,24,American Airlines,NaN,AA,AAL,AMERICAN,United States,Y,AA,24.0,LEX,4017.0,DFW,3670.0,Y,0,ERD ER4,N
4,24,American Airlines,NaN,AA,AAL,AMERICAN,United States,Y,AA,24.0,LEX,4017.0,ORD,3830.0,Y,0,ERD ER4,N
5,24,American Airlines,NaN,AA,AAL,AMERICAN,United States,Y,AA,24.0,ORD,3830.0,LEX,4017.0,Y,0,ERD ER4,N
6,35,Allegiant Air,NaN,G4,AAY,ALLEGIANT,United States,Y,G4,35.0,FLL,3533.0,LEX,4017.0,NaN,0,M80,N
7,35,Allegiant Air,NaN,G4,AAY,ALLEGIANT,United States,Y,G4,35.0,LEX,4017.0,FLL,3533.0,NaN,0,M80,N
8,35,Allegiant Air,NaN,G4,AAY,ALLEGIANT,United States,Y,G4,35.0,LEX,4017.0,PGD,7056.0,NaN,0,M80,N
9,35,Allegiant Air,NaN,G4,AAY,ALLEGIANT,United States,Y,G4,35.0,LEX,4017.0,PIE,3617.0,NaN,0,320,N


In [90]:
uniques = routes.drop_duplicates(['source', 'dest', 'codeshare'])
uniques[:200]

,airline,airline_id,source,source_id,dest,dest_id,codeshare,stops,equipment
0,2B,410.0,AER,2965.0,KZN,2990.0,NaN,0,CR2
1,2B,410.0,ASF,2966.0,KZN,2990.0,NaN,0,CR2
2,2B,410.0,ASF,2966.0,MRV,2962.0,NaN,0,CR2
3,2B,410.0,CEK,2968.0,KZN,2990.0,NaN,0,CR2
4,2B,410.0,CEK,2968.0,OVB,4078.0,NaN,0,CR2
5,2B,410.0,DME,4029.0,KZN,2990.0,NaN,0,CR2
6,2B,410.0,DME,4029.0,NBC,6969.0,NaN,0,CR2
7,2B,410.0,DME,4029.0,TGK,NaN,NaN,0,CR2
8,2B,410.0,DME,4029.0,UUA,6160.0,NaN,0,CR2
9,2B,410.0,EGO,6156.0,KGD,2952.0,NaN,0,CR2


In [98]:
uniq_outbound = uniques['source'].value_counts()
uniq_outbound
uniq_inbound = uniques['dest'].value_counts()
uniq_inbound
outlex = uniques['source'].str.count('LEX').sum()
outlex
intlex = uniques['dest'].str.count('LEX').sum()
intlex
rlexout=rports['source'].str.count('LEX').sum()
rlexout
rlexin=rports['dest'].str.count('LEX').sum()
rlexin

15

In [100]:
rlexout=rports['source'].str.count('LEX').sum()
rlexout
rlexin=rports['dest'].str.count('LEX').sum()
rlexin

15